# Divvy Re-Rebalancing Route Optimization Model

# Objective Functions & Constraints

#### Author: Xuan Amy Lin 
#### xxlin@uchicago.edu | M.S. in Analytics, University of Chicago | linkedin.com/in/xuanamylin/

<br>

The project aims to solve the classical traffic re-balancing problem in bike sharing systems. The dataset used is open-sourced by Divvy, the bike sharing program in Chicago.

This file introduces the objective function and constraints, as well as the constants and variables used. The problem is framed as a mixed-integer linear programming problem, because this type of problems guarantees one optimal solution, if any solution exists.

The solution of the problem and a demonstration on real-life case is coded in Julia [here](Hidden_landing_URL).

### Variables

$x_{i,j}$: A binary routing variable that indicates whether the van passes through station $i$ to $j$. For example, when $x_{1,3} = 1$, the van goes from station 1 to 3. If $x_{4,2}=0$, then the van does not travel from station 4 to 2.

$u_i$: The number of bikes on the van before arriving at station $i$

$d_i$: The number of bikes dropped off at station $i$

$p_i$: The number of bikes picked up at station $i$


### Constants

$N_i$: The ideal number of bikes after re-distribution at station $i$

$b_i$: The existing number of bikes at station $i$

$R_i$: The capacity or number of bike slots at station $i$

$C$: Capacity of the van

$t_{i,j}$: The amount of time required to travel from station $i$ to station $j$

$T$: The maximum period of time allowed to complete the entire trip

$S_i$: The variable indicating whether there is bike surplus at station $i$. In other words, $S_i = 1$ when $N_i > b_i$, and $S_i = 0$ when $N_i \leq b_i$. This is a dummy variable that helps to calculate absolute values.

$n$: The number of stations

### Assumptions

1. There is only one van distributing bikes.

2. Each station is visited once at most.

3. The van cannot drop off or pick up more than the ideal quantity at each station. For example, if N = 8 and b = 2, the van cannot drop off more than 6 bikes leaving N > 8.

4. The route starts from station 1 and ends at station 1. The van returns with zero bike.

5. The objective function is the sum of absolute values of errors. An error is defined as the difference between the ideal number of bikes (N) and the actual number of bikes after re-distribution (b + d - p).

### Objective Function

The objective function is **the number of dissatisfied customers**. 

A dissatisfied customer is defined as the customer who either finds no available bike to check out or finds no available rack to park the bike. In other words, having more than or less than the predicted number of bikes at a given station leads to a higher number of dissatisfied customers. 

With that said, the objective function is written as the absolute difference between predicted (expected) user demand and the actual user demand after optimization.

<br>

$$\min_{d_i, p_i} \quad \sum|N_i - (b_i + d_i - p_i)|$$

Now let's make it linear.

Since absolute values don't fulfill the requirement of linear programming, we rewrite the function into its equivalent below--a new function complemented by a constraint--with the help of the proxy variable S, the surplus variable.

$$\min_{d_i, p_i} \quad \sum_{i=1}^n S_i(b_i + d_i - p_i - N_i) + (1-S_i)(N_i - b_i - d_i + p_i)$$

s.t. $\quad$ For all station ($i$),  $S_i(b_i + d_i - p_i - N_i) + (1-S_i)(N_i - b_i - d_i + p_i) \geq 0)$

### Constaints

There are two clusters of constraints--routing constraints and capacity constraints. The routing constraints take care of the continuity of route while the capacity constraints is concerned with the number of bikes to be picked up or dropped off along that given route.

Some of the constaints are only applied to very specific scenarios, not to all scenarios, and it is achieved by the extensive of non-binding inequations. The intuitive explanations of these non-binding inequations are highlighted in blue.

##### Routing Constraints

* Same number of archs going in and going out

    For all stations $(i)$, $\quad \sum_{j=1}^n x_{i,j} = \sum_{j=1}^n x_{j,i}$

<br>

* All stations are passed once at most

    For all stations $(i)$, $\quad \sum_{j=1}^n x{i,j} \leq 1$

<br>

* Going from and to the same point is not permitted

    For all stations $(i)$, $\quad \sum_{i=1}^n x_{i,i} = 0$

<br>

* Going back and forth between 2 stations are not permitted; prevent situations where i to j and j to i are both travelled (2)

    For all pair of 2 stations ($i \neq j$), $\quad x_{i,j} + x_{j,i} \leq 1$

<br>

* Station 1 is the starting point and end point

    $\sum_{j=2}^n x_{1,j} = 1$

    $\sum_{i=2}^n x_{i,1} = 1$

<br>

* Traveled within the given time frame

    $\sum_{i=1}^n \sum_{j=1}^n x_{i,j}  t_{i,j} <= T$

<br>

* Ensure the continuity of route with a non-decreasing variable

    For all stations ($i$=1:n, $j$=2:n), $\quad nd_j \geq nd_i + 1 - 9999 (1-x_{i,j})$

    > For routes traveled,  $\quad nd_j \geq nd_i + 1$.
    
    > For routes not traveled, $\quad nd_j \geq nd_i - 9998$. Given that $nd_j \geq 0$, it's non-binding.

##### Capacity Constraints

<br>

* The van starts empty and zero bike could be dropped off from an empty van

    $u_1 = 0$

    $d_1 = 0$

<br>

* Bikes picked up and dropped off fit into the given station

    For all stations $(i)$, $\quad 0 \leq p_i \leq b_i$

    For all stations $(i)$, $\quad 0 \leq d_i \leq R_i - b_i$

<br>

* Total pick-ups <= drop-offs

    $\sum_{i=1}^n d_i \leq \sum_{i=1}^n p_i$

<br>

* Either drop off or pick up at any given station

    For all stations ($i$=2:n), $\quad S_i d_i + (1-S_i)p_i = 0$

    > If station has surplus ($S_i = 1$), $\quad d_i = 0$.</font>

    > Otherwise ($S_i = 0$), $\quad p_i = 0$.</font>

<br>

* Synergy with the route: only drop off and pick up along the route

    For all stations ($i$=2:n), $\quad u_i+d_i + p_i + nd_i \leq 999\sum_{j=1}^n x_{j,i}$
    
    > If the van DOESN'T traverse station $i$, $u_i + d_i + p_i + nd_i \leq 0$.

    > Otherwise, $\quad u_i + d_i + p_i + nd_i \leq 999$, non-binding.
    
    For all stations ($i$=2:n), $\quad d_i + p_i \geq \sum_{j=1}^n x_{j,i}$

    > If the van DOES traverse station $i$, $\quad d_i + p_i \geq 1$.

    > Otherwise, $\quad d_i + p_i \geq 0$, non-binding.
        
<br>

* Continuity of van inventory $u_i$

    Between two stations (i=1:n, j=2:n), $\quad u_j \geq u_i - d_i + p_i - C(1-x_{i,j})$

    $u_j \leq u_i - d_i + p_i - C(1-x_{i,j})$
    
    > If the van goes from $i$ to $j$, $u_j \geq u_i - d_i + p_i$.

    > Otherwise, $u_j \geq u_i - d_i + p_i - C$, non-binding.


